## Assigment of week 3.
## Segmenting and Clustering Neighborhoods in Toronto
### Part 3

In this notebook, I'll explore and cluster the neighborhoods in Toronto.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

## 1. Download and Explore Dataset

In previews notebook I worked preparing the dataframe with the information of Postal code of Toronto, with names of Borough, neigborhoods and latitude and longitude data.  

In [2]:
#read the file with the whole data
neighborhoods=pd.read_csv('toronto_data.csv')
neighborhoods.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with borough superimposed on top.

In [4]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now, let's segment and cluster only the neighborhoods in Central Toronto. So let's slice the original dataframe and create a new dataframe of the Central Toronto data.

In [5]:
centralT_data = neighborhoods[neighborhoods['Borough'] == 'Central Toronto'].reset_index(drop=True)
centralT_data.head()

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,47,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


Let's get the geographical coordinates of Central Toronto

In [6]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


As we did with all of Toronto City, let's visualizate Central Toronto the neighborhoods in it.

In [7]:
# create map of Central Toronto using latitude and longitude values
map_centralt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(centralT_data['Latitude'], centralT_data['Longitude'], centralT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_centralt)  
    
map_centralt

Next, we are going to start utilizing the Foursquare API to explore the borough and segment them.

In [8]:
#define Foursquare credentials
CLIENT_ID = 'SDB14ASDOOCN0HI0UDVMFPWSZOEKG5M1LOI5FNDTBKXNOIIB' # your Foursquare ID
CLIENT_SECRET = 'RYALKSQNP4PAX1U2LV31JMYJOQ2K2HG4YXHR5SUR0WCLWAHA' # your Foursquare Secret
VERSION = '20200603' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SDB14ASDOOCN0HI0UDVMFPWSZOEKG5M1LOI5FNDTBKXNOIIB
CLIENT_SECRET:RYALKSQNP4PAX1U2LV31JMYJOQ2K2HG4YXHR5SUR0WCLWAHA


#### Let's explore the first borough in our dataframe.

In [9]:
#Get the borough's name.
centralT_data.loc[0, 'Borough']

'Central Toronto'

Get the borough's latitude and longitude values.

In [10]:
borough_latitude = centralT_data.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = centralT_data.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = centralT_data.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of Central Toronto are 43.7280205, -79.3887901.


#### Now, let's get the top 100 venues that are in Central Toronto within a radius of 500 meters.

In [11]:
#First, let's create the GET request URL. Name your URL **url**.
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=SDB14ASDOOCN0HI0UDVMFPWSZOEKG5M1LOI5FNDTBKXNOIIB&client_secret=RYALKSQNP4PAX1U2LV31JMYJOQ2K2HG4YXHR5SUR0WCLWAHA&v=20200603&ll=43.7280205,-79.3887901&radius=500&limit=100'

Send the GET request and examine the resutls

In [12]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed7f47cf7706a001ba313ba'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


And how many venues were returned by Foursquare?

In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Central Toronto

#### Let's create a function to repeat the same process to all the borough in Central Toronto

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each borough and create a new dataframe called *centralT_venues*.

In [18]:
# type your answer here

centralT_venues = getNearbyVenues(names=centralT_data['Neighborhood'],
                                   latitudes=centralT_data['Latitude'],
                                   longitudes=centralT_data['Longitude']
                                  )


Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill Road Park, Forest Hill North & West
The Annex, North Midtown, Yorkville


#### Let's check the size of the resulting dataframe

In [19]:
print(centralT_venues.shape)
centralT_venues.head()

(114, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [20]:
centralT_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
109,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Tim Hortons,43.675800,-79.403532,Coffee Shop
110,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Subway,43.675650,-79.410255,Sandwich Place
111,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Nick's Shoe Repair,43.675457,-79.403675,Shoe Repair
112,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,The Kitchen Table,43.675770,-79.402746,Grocery Store
113,"The Annex, North Midtown, Yorkville",43.67271,-79.405678,Davenport Garden Centre,43.675851,-79.401894,Flower Shop


Let's check how many venues were returned for each neighborhood

In [21]:
centralT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,32,32,32,32,32,32
Davisville North,9,9,9,9,9,9
"Forest Hill Road Park, Forest Hill North & West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",4,4,4,4,4,4
"North Toronto West, Lawrence Park",20,20,20,20,20,20
Roselawn,3,3,3,3,3,3
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",23,23,23,23,23,23


#### Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(centralT_venues['Venue Category'].unique())))

There are 66 uniques categories.


## 3. Analyze Each Neighborhood

In [23]:
# one hot encoding
centralT_onehot = pd.get_dummies(centralT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centralT_onehot['Neighborhood'] = centralT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [centralT_onehot.columns[-1]] + list(centralT_onehot.columns[:-1])
centralT_onehot = centralT_onehot[fixed_columns]

centralT_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#And let's examine the new dataframe size.
centralT_onehot.shape

(114, 67)

In [25]:
centralT_grouped = centralT_onehot.groupby('Neighborhood').mean().reset_index()
centralT_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0000,0.000000,0.0000,0.0000,0.000000,0.03125,0.000000,0.000000,0.062500,...,0.0000,0.0625,0.000000,0.00,0.03125,0.03125,0.00,0.000000,0.0000,0.00
1,Davisville North,0.0000,0.000000,0.0000,0.0000,0.111111,0.00000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.00,0.00000,0.00000,0.00,0.000000,0.0000,0.00
2,"Forest Hill Road Park, Forest Hill North & West",0.0000,0.000000,0.0000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0000,0.2500,0.000000,0.00,0.00000,0.00000,0.25,0.000000,0.0000,0.00
3,Lawrence Park,0.0000,0.000000,0.0000,0.0000,0.000000,0.00000,0.000000,0.333333,0.000000,...,0.0000,0.0000,0.333333,0.00,0.00000,0.00000,0.00,0.000000,0.0000,0.00
4,"Moore Park, Summerhill East",0.0000,0.000000,0.0000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.25,0.00000,0.00000,0.00,0.000000,0.0000,0.00
5,"North Toronto West, Lawrence Park",0.0000,0.000000,0.0500,0.0000,0.000000,0.00000,0.000000,0.000000,0.050000,...,0.0000,0.0000,0.000000,0.00,0.00000,0.00000,0.00,0.000000,0.0000,0.05
6,Roselawn,0.0000,0.000000,0.0000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0000,0.0000,0.000000,0.00,0.00000,0.00000,0.00,0.000000,0.0000,0.00
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.0625,0.000000,0.0625,0.0625,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.0625,0.0625,0.000000,0.00,0.00000,0.00000,0.00,0.000000,0.0625,0.00
8,"The Annex, North Midtown, Yorkville",0.0000,0.043478,0.0000,0.0000,0.000000,0.00000,0.043478,0.000000,0.130435,...,0.0000,0.0000,0.000000,0.00,0.00000,0.00000,0.00,0.043478,0.0000,0.00


In [26]:
# Let's confirm the new size
centralT_grouped.shape

(9, 67)

#### Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in centralT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = centralT_grouped[centralT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2         Pizza Place  0.06
3                Café  0.06
4  Italian Restaurant  0.06


----Davisville North----
               venue  freq
0        Pizza Place  0.11
1              Hotel  0.11
2   Department Store  0.11
3       Dance Studio  0.11
4  Food & Drink Shop  0.11


----Forest Hill Road Park, Forest Hill North & West----
                 venue  freq
0                 Park  0.25
1        Jewelry Store  0.25
2                Trail  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1          Swim School  0.33
2             Bus Line  0.33
3  Rental Car Location  0.00
4                  Pub  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Restaurant  0.25
1                 Park  0.25
2           Playground  0.25
3         Tennis Court  0.25
4  Rental C

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = centralT_grouped['Neighborhood']

for ind in np.arange(centralT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(centralT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Pizza Place,Café,Sushi Restaurant,Coffee Shop,Farmers Market,Restaurant
1,Davisville North,Park,Gym / Fitness Center,Hotel,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Yoga Studio
2,"Forest Hill Road Park, Forest Hill North & West",Trail,Jewelry Store,Park,Sushi Restaurant,Yoga Studio,Garden,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
3,Lawrence Park,Bus Line,Park,Swim School,Gas Station,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Yoga Studio
4,"Moore Park, Summerhill East",Restaurant,Playground,Tennis Court,Park,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Garden
5,"North Toronto West, Lawrence Park",Clothing Store,Coffee Shop,Yoga Studio,Ice Cream Shop,Metro Station,Mexican Restaurant,Fast Food Restaurant,Park,Diner,Rental Car Location
6,Roselawn,Ice Cream Shop,Garden,Pool,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Yoga Studio,Donut Shop
7,"Summerhill West, Rathnelly, South Hill, Forest...",Pub,Coffee Shop,American Restaurant,Supermarket,Restaurant,Liquor Store,Light Rail Station,Fried Chicken Joint,Sports Bar,Pizza Place
8,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Indian Restaurant,Shoe Repair,History Museum,Flower Shop,Liquor Store,Middle Eastern Restaurant,Park


## 4. Cluster Neighborhoods


Run *k*-means to cluster the neighborhood into 3 clusters.

In [50]:
# set number of clusters
kclusters = 3

centralT_grouped_clustering = centralT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(centralT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 2, 0, 0, 1, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

centralT_merged = centralT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
centralT_merged = centralT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

centralT_merged.head() # check the last columns!

,Unnamed: 0,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Gas Station,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Yoga Studio
1,45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Park,Gym / Fitness Center,Hotel,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Yoga Studio
2,46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Ice Cream Shop,Metro Station,Mexican Restaurant,Fast Food Restaurant,Park,Diner,Rental Car Location
3,47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Pizza Place,Café,Sushi Restaurant,Coffee Shop,Farmers Market,Restaurant
4,48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Restaurant,Playground,Tennis Court,Park,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Garden


Finally, let's visualize the cluster

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(centralT_merged['Latitude'], centralT_merged['Longitude'], centralT_merged['Neighborhood'], centralT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### Cluster 1

In [53]:
centralT_merged.loc[centralT_merged['Cluster Labels'] == 0, centralT_merged.columns[[1] + list(range(5, centralT_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4P,-79.390197,0,Park,Gym / Fitness Center,Hotel,Pizza Place,Breakfast Spot,Sandwich Place,Department Store,Dance Studio,Food & Drink Shop,Yoga Studio
2,M4R,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Ice Cream Shop,Metro Station,Mexican Restaurant,Fast Food Restaurant,Park,Diner,Rental Car Location
3,M4S,-79.388790,0,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Pizza Place,Café,Sushi Restaurant,Coffee Shop,Farmers Market,Restaurant
4,M4T,-79.383160,0,Restaurant,Playground,Tennis Court,Park,Fried Chicken Joint,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Garden
5,M4V,-79.400049,0,Pub,Coffee Shop,American Restaurant,Supermarket,Restaurant,Liquor Store,Light Rail Station,Fried Chicken Joint,Sports Bar,Pizza Place
7,M5P,-79.411307,0,Trail,Jewelry Store,Park,Sushi Restaurant,Yoga Studio,Garden,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
8,M5R,-79.405678,0,Café,Sandwich Place,Coffee Shop,Indian Restaurant,Shoe Repair,History Museum,Flower Shop,Liquor Store,Middle Eastern Restaurant,Park


#### Cluster 2

In [54]:
centralT_merged.loc[centralT_merged['Cluster Labels'] == 1, centralT_merged.columns[[1] + list(range(5, centralT_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M5N,-79.416936,1,Ice Cream Shop,Garden,Pool,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Yoga Studio,Donut Shop


#### Cluster 3

In [55]:
centralT_merged.loc[centralT_merged['Cluster Labels'] == 2, centralT_merged.columns[[1] + list(range(5, centralT_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,-79.38879,2,Bus Line,Park,Swim School,Gas Station,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Yoga Studio
